In [7]:
library(NELSI)
library(TreeSim)
library(TreePar)
source('02-Ebola-Functions.R')

In [4]:
trees <- read.tree('02-EbolaTrees.trees')

In [5]:
# Skyline
# estimate death rate:
deathfix <- 0
sprobc <- 0.7
# 3 Re intervals
numbRe <- 3

In [6]:
# Calculate likelihood for first 5 trees
numbTree <- 4

In [23]:
likdiff <- vector()
estimates <- c()
for(index in 1:numbTree){
    test <- trees[[index]]
    droptip<-c("EBOV|KM034550|EM095|SierraLeone_EM|2014-05-25","EBOV|KM034554|G3676|SierraLeone_G|2014-05-27","EBOV|KM034559|G3680|SierraLeone_G|2014-05-28","EBOV|KM034563|G3687|SierraLeone_G|2014-05-28","EBOV|KM034561|G3683|SierraLeone_G|2014-05-28","EBOV|KM034562|G3686|SierraLeone_G|2014-05-28")
    test<-drop.tip(test,droptip)
    rootheight<-max(getx(test,sersampling=1)[,1])
    x<-getx(test,sersampling=1)
    times<-x[,1]
    ttype<-x[,2]
    
    if (numbRe == 1){
        out <- optim(c(2, 1, 0.7), LikShiftsSTTebolaConst, times = times, 
                    ttype =ttype, sprobc = sprobc, 
                    deathfix =deathfix, cutoff = 0)
    }else{
       out<-optim(c(rep(2,numbRe),1,0.7),LikShiftsSTTebola,
                  times=times,ttype=ttype,sprobc=sprobc,
                  deathfix=deathfix,cutoff=0)
    }
    estimates <- rbind(estimates, parepi(out, sprobc=sprobc, 
                                        deathfix=deathfix))
}

In [31]:
estimates

1.7939327,1.3311000,0.6782099,4.2882630,0.7000000
1.854099,1.871515,1.628523,6.973842,0.700000
1.5267441,0.7939055,1.0804592,3.2563608,0.7000000
1.5858334,2.0635427,0.5375628,6.4930770,0.7000000


In [32]:
estimatesraw<-estimates

estimatesmedian<-vector()
estimatesHPD<-vector()
estimatesmean<-vector()
estimatesvar<-vector()
for (i in 1:length(estimates[1,])){
    estimatesmedian<-c(estimatesmedian,median(estimates[,i]))
    estimatesHPD<-cbind(estimatesHPD,HPD(estimates[,i]))
    estimatesmean<-c(estimatesmean,mean(estimates[,i]))
    estimatesvar<-c(estimatesvar,var(estimates[,i]))
}

In [33]:
estimates<-round(rbind(estimatesmedian,estimatesHPD,estimatesmean,estimatesvar,estimatesraw),2)
colnames(estimates)<-c("R1","R2","R3","rateUninf","sampProb")
rownames(estimates)<-c("median","95low","95up","mean","var",1:numbTree)
print(estimates)

         R1   R2   R3 rateUninf sampProb
median 1.69 1.60 0.88      5.39      0.7
95low  1.53 1.33 0.54      4.29      0.7
95up   1.79 2.06 1.08      6.97      0.7
mean   1.69 1.52 0.98      5.25      0.7
var    0.03 0.33 0.24      3.14      0.0
1      1.79 1.33 0.68      4.29      0.7
2      1.85 1.87 1.63      6.97      0.7
3      1.53 0.79 1.08      3.26      0.7
4      1.59 2.06 0.54      6.49      0.7
